## Part 4 - Deploy Model

**Connect to Azure ML workspace and get deployed model**

In [1]:
from azureml.core import Workspace
from azureml.core import Model

workspace = Workspace.from_config()
model = Model(workspace, name='cifar-classifier')

**Define score file and environment (inference configuration)**

In [3]:
%pycat deploy/score.py

In [4]:
%pycat deploy/env.yml

In [6]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='deploy',
                                   runtime='python',
                                   entry_script='score.py',
                                   conda_file='env.yml')

**Deploy as local web service**

In [9]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import LocalWebservice

local_service_name = 'local-cifar-classifier'
local_config = LocalWebservice.deploy_configuration(port=3000)

try:
    local_service = Webservice(workspace, name=local_service_name)
    if local_service:
        local_service.delete()
except WebserviceException as e:
    print()

local_service = Model.deploy(workspace, local_service_name, [model], inference_config, local_config)
local_service.wait_for_deployment()


Generating Docker build context.
2020/02/19 05:55:29 Downloading source code...
2020/02/19 05:55:30 Finished downloading source code
2020/02/19 05:55:31 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/19 05:55:33 Successfully set up Docker network: acb_default_network
2020/02/19 05:55:33 Setting up Docker configuration...
2020/02/19 05:55:34 Successfully set up Docker configuration
2020/02/19 05:55:34 Logging in to registry: johneast3c830537.azurecr.io
2020/02/19 05:55:40 Successfully logged into johneast3c830537.azurecr.io
2020/02/19 05:55:40 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/19 05:55:40 Scanning for dependencies...
2020/02/19 05:55:41 Successfully scanned dependencies
2020/02/19 05:55:41 Launching container with name: acb_step_0
Sending build context to Docker daemon  60.93kB
Step 1/15 : FROM mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04@sha256:a1b514f3ba884b9a7695cbba56

  Created wheel for pycparser: filename=pycparser-2.19-py2.py3-none-any.whl size=111031 sha256=618773f784a014e56948fed4267fcbe038c805e67581054205ec575efe9e5294
  Stored in directory: /root/.cache/pip/wheels/c6/6b/83/2608afaa57ecfb0a66ac89191a8d9bad71c62ca55ee499c2d0
Successfully built wrapt json-logging-py liac-arff dill pycparser


#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_01e6b924c5f1d28e8804005fb447330c
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 45d512740135
 ---> 31d3dcbc8f5c
Step 9/15 : ENV PATH /azureml-envs/azureml_01e6b924c5f1d28e8804005fb447330c/bin:$PATH
 ---> Running in e4d8071d7a31
Removing intermediate container e4d8071d7a31
 ---> 3be015bcdaec
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_01e6b924c5f1d28e8804005fb447330c
 ---> Running in f5250a29fae9
Removing intermediate container f5250a29fae9
 ---> 999c93f83619
Step 11/15 : ENV LD_LIBRARY_PATH /azur

**Test local service**

In [12]:
from io import BytesIO
import urllib.request
import base64
import io
import requests 
import json
import argparse
from PIL import Image

def imgToBase64(img):
    '''Convert pillow image to base64-encoded image'''
    imgio = BytesIO()
    img.save(imgio, 'JPEG')
    img_str = base64.b64encode(imgio.getvalue())
    return img_str.decode('utf-8')

def test_service(image_url, scoring_url):
    # Download image and convert to base 64
    with urllib.request.urlopen(image_url) as url:
        test_img = io.BytesIO(url.read())

    base64Img = imgToBase64(Image.open(test_img))
    
    # Get prediciton through endpoint
    input_data = '{\"data\": \"'+ base64Img +'\"}'
    headers = {'Content-Type':'application/json'}
    response = requests.post(scoring_url, input_data, headers=headers)
    return json.loads(response.text)

In [17]:
scoring_url = local_service.scoring_uri
image_url = 'https://content.presspage.com/uploads/2431/1920_cairoa380new-135207.jpg?10000'

prediction = test_service(image_url, scoring_url)
print(prediction)

{"label": "airplane", "probability": "0.9778827"}
